In [0]:
import pandas as pd
import numpy as np
import scipy as sp
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from copy import deepcopy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import random as rd
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import NearestNeighbors
nltk.download('stopwords')

Importing data...

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_csv("/content/drive/My Drive/Avondale_Restaurant_Review.csv", delimiter=",", engine='python')

In [0]:
def preprocess_review(review):
    """Helper function to clean the reviews.

     Arg: review: review text.
     Returns: clean_review : Cleaned reviews

     You should carry out the following steps.
     1. Remove HTML Tags.
     2. Remove non-letter characters.
     3. Convert to lower case.
     4. Remove stopwords.
    """

    #Write your code below.
    
    ## HTML Tags removal
    clean1=BeautifulSoup(review)  
    
    ## Non-letter characters removal
    clean2=re.sub("[^a-zA-Z]", " ", clean1.get_text()) 
    
    ## Converting to lower case and splitting each word
    clean3=clean2.lower().split()
    
    ## Removing stopwords
    stops = set(stopwords.words("english"))
    clean4=[w for w in clean3 if not w in stops]
    
    ## Returning the reviews with spaces between words
    clean_review=" ".join(clean4)

    
    return clean_review

Splitting >=4 and 4 out of 5 as good (1) and NOT good (0).

In [0]:
data.loc[data['stars'] > 4, 'Good Review'] = 1
data.loc[data['stars'] <= 4, 'Good Review'] = 0
print(sum(data['Good Review'] == 0) / len(data['Good Review']) * 100, 'percent of reviews are bad (less than 5 star).')

58.600537039962084 percent of reviews are bad (less than 5 star).


In [0]:
def preprocess_review(review):
    """Helper function to clean the reviews.

     Arg: review: review text.
     Returns: clean_review : Cleaned reviews

     You should carry out the following steps.
     1. Remove HTML Tags.
     2. Remove non-letter characters.
     3. Convert to lower case.
     4. Remove stopwords.
    """

    #Write your code below.
    
    ## HTML Tags removal
    clean1=BeautifulSoup(review)  
    
    ## Non-letter characters removal
    clean2=re.sub("[^a-zA-Z]", " ", clean1.get_text()) 
    
    ## Converting to lower case and splitting each word
    clean3=clean2.lower().split()
    
    ## Removing stopwords
    stops = set(stopwords.words("english"))
    clean4=[w for w in clean3 if not w in stops]
    
    ## Returning the reviews with spaces between words
    clean_review=" ".join(clean4)

    
    return clean_review

80-20 split on the data and then cleaning the reviews based on method described in HW1.

In [0]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]

#Clean the reviews and add them to the list below
reviews_train_clean = []
reviews_test_clean = []
#Write your code below.
num_reviews = train["text"].size
tlist=train["text"].tolist()
for i in range(num_reviews):
  cr=preprocess_review(tlist[i])
  reviews_train_clean.append(cr)

num_reviews = test["text"].size
tlist=test["text"].tolist()
for i in range(num_reviews):
    cr=preprocess_review(tlist[i])
    reviews_test_clean.append(cr)    

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Formed design matrices: Conts, Binary and TFIDF...

In [0]:
def design_matrix(cleaned_reviews, cv):

    data_features = cv.transform(cleaned_reviews)

    X_counts = data_features.toarray()

    X_binary=X_counts

    X_binary[X_binary>0]=1

    transformer = TfidfTransformer()

    X_tfidf = transformer.fit_transform(X_counts).toarray()
   
    return X_counts,X_binary,X_tfidf

In [0]:
cv = CountVectorizer(analyzer = "word",   \
                            tokenizer = None,    \
                            preprocessor = None, \
                            stop_words = None,   \
                            max_features = 500) 

cv.fit(reviews_train_clean)

X_train_counts,X_train_binary,X_train_tfidf = design_matrix(reviews_train_clean, cv) 
X_test_counts,X_test_binary,X_test_tfidf = design_matrix(reviews_test_clean, cv)

In [0]:
from warnings import filterwarnings
filterwarnings('ignore')
import time

def calculateF1(X, y, k = 5):
    """calculateF1(X, y, k = 5) return two list which record all randomly selected c(in the interval (1e-4, 1e4))
     and corresponding F1 scores.

     Args: X: Features
           y: Label of sentiment
           k: Number of Cross-validation

     Returns: c_list: List of all c values.
              f1_list: Corresponding F1 Scores.
    """
    rd.seed(0) #Setting a common seed

    #Write your code here.
    start=time.time()
    c_list=[10**i for i in np.random.uniform(-4, 4, 30)]
    f1_list=[]
    kf = KFold(n_splits=k)
    print('itr# Time')
    i=0
    for c in c_list:
        f1score=0.0
        i+=1
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf = LogisticRegression(C=c)
            clf = clf.fit(X_train, y_train)
            y_pred_test = clf.predict(X_test)
            f1score+=f1_score(y_test, y_pred_test)/float(k)
        f1_list.append(f1score)
        if i%5==0:
            print(i, time.time()-start)        
        
    return c_list, f1_list

In [0]:
def findBestC(X, y, k = 5):
    """findBestC(X, y, k) return the best performance c, and the improvement(difference between best and worst f1_scores)/
     Args: X: Features
           y: Label of sentiment
           k: Number of Cross-validation
     Returns: c_best: C value with best f1_score.
              improvement: difference between best and worst f1_score.
    """
    #Write your code here. 
    c_list, f1_list = calculateF1(X, y, k)
    f1max = max(f1_list)
    c_best = c_list[f1_list.index(f1max)]
    improvement = f1max - min(f1_list)
    return c_best,improvement

In [0]:
def findImprovement(X,train_sentiment,test_size = 0.2, random_state = 0):
    """ Find the improvement in F1-Score of the design Matrix(X) using previous utility functions and the test_f1_score using the best C.

      Args: X: Design Matrix
            train_sentiment: Sentiments of the training data
            test_size: Split it as 80:20
            random_state: Seed

      Returns:
            c_best: The best possible c value
            improvement: improvement in F1-Score using the design Matrix(X).
            f1_s: Test F1 Score.
            

      You should carry out the following Steps:
      1. Split the data using the above parameters.
      2. Find out the best c and the improvement. (use 5-fold Cross Validation.)
      3. Find out the test f1 score with this c.
    """
    #Write your code here.
    X_train, X_test, y_train, y_test = train_test_split(X, train_sentiment, test_size=test_size, random_state=random_state)
    c_best, improvement = findBestC(X_train, y_train)
    clf = LogisticRegression(C=c_best)
    clf = clf.fit(X_train, y_train)
    y_pred_test = clf.predict(X_test)
    f1_s = f1_score(y_test, y_pred_test)   

    return c_best,improvement,f1_s

In [0]:
target_train = train['Good Review'].values
target_test = test['Good Review'].values
c_best,improvement,f1_s = findImprovement(X_train_counts, target_train)
print('best c:', c_best, '; improvement:', improvement, '; test f1_score:', f1_s)


itr# Time
5 4.9772186279296875
10 10.12351393699646
15 14.22546672821045
20 18.082712650299072
25 22.64530301094055
30 28.434545755386353
best c: 0.26647371234970335 ; improvement: 0.7538168063838642 ; test f1_score: 0.7448609431680774


In [0]:
clf = LogisticRegression(C=c_best)
clf = clf.fit(X_train_counts, target_train)
target_test_pred_counts = clf.predict(X_test_counts)
print('Counts Accuracy:', f1_score(target_test, target_test_pred_counts))

Counts Accuracy: 0.7467312348668281


# ***Using KNN Classifier ***

In [0]:
from warnings import filterwarnings
from sklearn.neighbors import KNeighborsClassifier
filterwarnings('ignore')
import time

def calculateF1(X, y, k = 2):
    """calculateF1(X, y, k = 5) return two list which record all randomly selected c(in the interval (1e-4, 1e4))
     and corresponding F1 scores.

     Args: X: Features
           y: Label of sentiment
           k: Number of Cross-validation

     Returns: c_list: List of all c values.
              f1_list: Corresponding F1 Scores.
    """
    rd.seed(0) #Setting a common seed

    #Write your code here.
    start=time.time()
    #c_list=[10**i for i in np.random.uniform(-4, 4, 30)]
    #c_list=[10,20,30,40,50,70,90,100,200]
    c_list=[10,20,40,70,100,200]
    f1_list=[]
    kf = KFold(n_splits=k)
    print('itr# Time')
    i=0
    for c in c_list:
        f1score=0.0
        i+=1
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            # model = KNeighborsClassifier(n_neighbors=3)
            # model.fit(X_train,y_train)
            # y_pred_test =model.predict(X_test)
            # f1_s = f1_score(y_test, y_pred_test)   
            clf=KNeighborsClassifier(n_neighbors=c)
            clf = clf.fit(X_train, y_train)
            y_pred_test = clf.predict(X_test)
            f1score+=f1_score(y_test, y_pred_test)/float(k)
        f1_list.append(f1score)
        if i%1==0:
            print(i, time.time()-start)        
        
    return c_list, f1_list

In [0]:
def findBestC(X, y, k = 2):
    """findBestC(X, y, k) return the best performance c, and the improvement(difference between best and worst f1_scores)/
     Args: X: Features
           y: Label of sentiment
           k: Number of Cross-validation
     Returns: c_best: C value with best f1_score.
              improvement: difference between best and worst f1_score.
    """
    #Write your code here. 
    c_list, f1_list = calculateF1(X, y, k)
    f1max = max(f1_list)
    c_best = c_list[f1_list.index(f1max)]
    improvement = f1max - min(f1_list)
    return c_best,improvement

In [0]:
def findImprovement(X,train_sentiment,test_size = 0.2, random_state = 0):
    """ Find the improvement in F1-Score of the design Matrix(X) using previous utility functions and the test_f1_score using the best C.

      Args: X: Design Matrix
            train_sentiment: Sentiments of the training data
            test_size: Split it as 80:20
            random_state: Seed

      Returns:
            c_best: The best possible c value
            improvement: improvement in F1-Score using the design Matrix(X).
            f1_s: Test F1 Score.
            

      You should carry out the following Steps:
      1. Split the data using the above parameters.
      2. Find out the best c and the improvement. (use 5-fold Cross Validation.)
      3. Find out the test f1 score with this c.
    """
    #Write your code here.
    X_train, X_test, y_train, y_test = train_test_split(X, train_sentiment, test_size=test_size, random_state=random_state)
    c_best, improvement = findBestC(X_train, y_train)
    clf=KNeighborsClassifier(n_neighbors=c_best)
    clf = clf.fit(X_train, y_train)
    y_pred_test = clf.predict(X_test)
    f1_s = f1_score(y_test, y_pred_test)   

    return c_best,improvement,f1_s

In [0]:
target_train = train['Good Review'].values
target_test = test['Good Review'].values
c_best,improvement,f1_s = findImprovement(X_train_counts, target_train)
print('best c:', c_best, '; improvement:', improvement, '; test f1_score:', f1_s)

itr# Time
1 369.9298598766327
2 738.7925205230713
3 1109.9048051834106
4 1484.1028571128845
5 1861.7807123661041
6 2235.4009606838226
best c: 40 ; improvement: 0.06075199138192122 ; test f1_score: 0.6997929606625259


In [0]:
clf=KNeighborsClassifier(n_neighbors=c_best)
clf = clf.fit(X_train_counts, target_train)
target_test_pred_counts = clf.predict(X_test_counts)
print('Counts Accuracy:', f1_score(target_test, target_test_pred_counts))

Counts Accuracy: 0.6887206058190514


# ***KNN finished***

In [0]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), clf.coef_[0]
    )
}
print('Good words:', end = "\n\n")
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
print('')
print('Bad words:', end = "\n\n")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)

Good words:

('amazing', 1.1231362991731006)
('delicious', 0.8682917821078282)
('thank', 0.7455727848180447)
('incredible', 0.7081842348265955)
('best', 0.7024155073245454)

Bad words:

('worst', -1.1174352432449874)
('ok', -0.9557728026765702)
('rude', -0.9388451010331978)
('horrible', -0.9195265109504851)
('bland', -0.8674215575962614)


In [0]:
c_best,improvement,f1_s = findImprovement(X_train_tfidf, target_train)
print('best c:', c_best, '; improvement:', improvement, '; test f1_score:', f1_s)

itr# Time
5 58.34920382499695
10 119.7146646976471
15 168.73808670043945
20 269.6752185821533
25 337.96801376342773
30 444.0407249927521
best c: 1.5433124001908993 ; improvement: 0.7556461423661043 ; test f1_score: 0.7592592592592593


In [0]:
clf = LogisticRegression(C=c_best)
clf = clf.fit(X_train_tfidf, target_train)
target_test_pred_tfidf = clf.predict(X_test_tfidf)
print('TDIDF Accuracy:', f1_score(target_test, target_test_pred_tfidf))

TDIDF Accuracy: 0.7647336162187647


# Creating Clusters...


In [0]:
import nltk.data
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]

#Clean the reviews and add them to the list below
reviews_train_clean = []
reviews_test_clean = []
#Write your code below.
num_reviews = train["text"].size
tlist=train["text"].tolist()
for i in range(num_reviews):
  cr=preprocess_review(tlist[i])
  reviews_train_clean.append(cr)

num_reviews = test["text"].size
tlist=test["text"].tolist()
for i in range(num_reviews):
    cr=preprocess_review(tlist[i])
    reviews_test_clean.append(cr)    

Creating reviews based on unique users

In [0]:
unique_id=np.unique(train['user_id'])
print( train)
# print(unique_id)
# print(train['text'][0])
user_based_reviews=[]
counter=0
print(train.loc[10000])
# for uid in unique_id:
#   id=np.where(train['user_id']==uid)
#   print(counter, id)
#   rev=""
#   for i in range(len(id[0])):
#     print("inside",i)
#     print(train['text'][id[0][i]])
#     rev=rev+" "+(train['text'][id[0][i]]) 
#   user_based_reviews.append([uid,rev])
#   counter+=1
  
# print(rev)
#print(train[np.where(train['user_id'] == 'uFVAAe0JC81IPmxgT49Hcw')])
# for uid in unique_id:
#   print(uid, train[uid])


                    review_id  ... Good Review
0      hUUfzKeDLo930LA9-v2tWQ  ...         1.0
1      c4t3x4G2goB2OuqrEiBzJA  ...         1.0
2      ZZXyJhzMoY1SFvyp5bRGgw  ...         0.0
3      MVB0A9akvrnz-ICyEfPFgQ  ...         0.0
4      SOSBVld0AsaO-uFULble1Q  ...         1.0
...                       ...  ...         ...
12656  s01nkwKtmAzZc9CbnZiqMQ  ...         0.0
12657  OU0-l3w4CcItPx35guOaJA  ...         1.0
12658  1FvT4lC86tahz4kQUpujGQ  ...         0.0
12660  I3LVHf4Y493-igtIzPOEjg  ...         0.0
12661  _85TjsM5SFTfGp7J66o1tw  ...         0.0

[10079 rows x 12 columns]


KeyError: ignored

Creating reviews based on Restaurants 

In [0]:
unique_id=np.unique(train['restaurant_name'])
print(unique_id.shape)
pri
print(train['restaurant_name'][0])
print( np.where(train['restaurant_name']=='Eest Asian Bistro'))
user_based_reviews=[]
counter=0
for uid in unique_id:
  id=np.where(train['restaurant_name']==uid)
  print(counter, id)
  rev=""
  for i in range(len(id[0])):
    print("inside",i)
    print(train['text'][id[0][i]])
    rev=rev+" "+(train['text'][id[0][i]]) 
  user_based_reviews.append([uid,rev])
  counter+=1
  
# print(rev)
#print(train[np.where(train['user_id'] == 'uFVAAe0JC81IPmxgT49Hcw')])
# for uid in unique_id:
#   print(uid, train[uid])


(163,)
Eest Asian Bistro
(array([  0,   4,  14,  16,  19,  25,  29,  35,  36,  38,  46,  49,  51,
        57,  60,  61,  62,  80,  85,  88,  98, 111, 128, 131, 137, 138,
       150, 153, 156, 157, 181, 193, 195, 198, 204, 218, 222, 225, 228,
       233, 234, 238, 244, 250, 264, 267, 271, 273, 276, 279, 289, 301,
       311, 322, 324, 327, 340, 347, 354, 359, 370, 384, 404, 409, 424,
       425, 427, 429, 432, 446, 462, 466, 470, 471, 472, 476, 484, 486,
       491, 495, 504, 506, 509, 511, 516, 520, 529, 531, 541, 560, 564,
       572, 576, 579, 580, 581, 592, 606, 610, 618, 623, 631, 637, 640,
       644, 646, 654, 662, 671, 675, 676, 687, 691, 695, 699, 700, 710,
       721, 726, 736, 740, 741, 743, 756, 764]),)
0 (array([ 10,  23,  43,  79,  86,  91, 104, 113, 115, 172, 175, 179, 206,
       211, 268, 278, 350, 355, 357, 373, 416, 431, 459, 474, 482, 505,
       552, 553, 557, 578, 600, 608, 626, 630, 647, 656, 661, 664, 683,
       707, 734, 737, 751, 771]),)
inside 0
Worst experie

KeyError: ignored

In [0]:
def clean_review(review, remove_stopwords = False):
    """Helper function to clean the reviews i.e. to convert a document to a sequence of words.
     Please note that we're not removing stopwords since word2vec relies on the broader context
     of the sentence in order to produce high-quality word vectors.

     Arg: review: review string (str)
          remove_stopwards: If true remove stopwords else not. (boolean)
     Returns: cleaned_review : Cleaned review (list)

     You should carry out the following steps.
     1. Remove HTML Tags.
     2. Remove non-letter characters.
     3. Convert to lower case.
    """
    ### Add your code here.
    review_without_html = BeautifulSoup(review).get_text()
    review_only_lettes = re.sub("[^a-zA-Z]"," ", review_without_html)
    cleaned_review = review_only_lettes.lower().split()
    #####################
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        cleaned_review = [w for w in cleaned_review if not w in stops]
    return cleaned_review

In [0]:
def review_to_sentences( review: str, tokenizer: nltk.tokenize.punkt.PunktSentenceTokenizer ):
    """Helper function to split a review into parsed sentences. Returns a 
     list of sentences, where each sentence is a list of words.

     Arg: review: review string (str)
          tokenizer: punkt tokenizer
     Returns:
          review_sentences: List of list of tokens.
                            e.g. [["word2vec", "was", "introduced", "by", "google" ],["it","leverages","distributed","token","representations"]]

     You should carry out the following steps.
     1. Use the tokenizer to split the paragraph into sentences.
     2. Clean the sentence to return a list of words for each sentence using the helper funtion above.
     3. Return a list of tokenized sentences.
    """
    ### Add your code here.
    tokenized_sentences = tokenizer.tokenize(review.strip())
    review_sentences = []
    for sentence in tokenized_sentences:
        if len(sentence) > 0:
            review_sentences.append( clean_review( sentence, remove_stopwords=True ))
    ######################
    
    return review_sentences

In [0]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
### Add your code here.
for review in reviews_train_clean:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


In [0]:
from gensim.models import word2vec, KeyedVectors

In [0]:
def generate_z1(sentences,num_features,min_word_count,context,num_workers = 4,downsampling = 1e-3, model_name = "model1_100features_40minwords_5context"):
    """Set values for the parameters of the your word2vec model and train it on the extracted sentences from both the train and unlabeled_train data and
      generate the collection of these representations Z1.
      You should carry out the following.
      1) Set the parameters as mentioned below. 
        A)Constrained Paramters : "context length", "embedding dimension", "min_words" (Please check the question for the values.)
        B)Optional Parameters: "number of workers", "downsample setting"
      2) Train your word2vec model and save it.
      3) Store the collection of word embeddings and the word_list(z1 and word_list_z1) .

      Arg: sentences: List of tokenized sentences (List)
            num_features: Word vector dimensionality (int)
            min_word_count: Minimum word count (int)
            context: Context window size (int)
            num_workers: Number of threads to run in parallel (int)
            downsampling: Downsample setting for frequent words (float)
            model_name = Name to save your model (str)
      Returns:
            trained_word2vec_model: word2vec model trained on the tokenized sentences.
            z1: word embeddings (ndarray)
            word_list_z1: List of tokens in the model (List)

      

    """
    print("Training model...")
    ### Add your code here.
    trained_word2vec_model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)
    trained_word2vec_model.save(model_name)

    #######################
    # word_list_z1 = set(model.index2word)
    word_list_z1 = trained_word2vec_model.wv.index2word
    #print(len(word_list_z1))
    z1 = trained_word2vec_model.wv[word_list_z1]
    # print("shape",z1.shape)
    # words = w2v_model.wv.index2word
    return trained_word2vec_model, z1, word_list_z1
    
# Set values for the parameters of the your word2vec model and train it on the extracted sentences from both the train and unlabeled_train data and
# generate the collection of these representations Z1.
# Add the function parameter values below.
### Add your code here.
num_features=100
min_word_count=40
context=5



######################
# generate_z1(sentences,num_features,min_word_count,context)

model1, z1, word_list_z1 = generate_z1(sentences,num_features,min_word_count,context)
print(len(word_list_z1))
print("shape",z1.shape)


In [0]:
from sklearn.cluster import KMeans
import time

In [0]:
def fit_kmeans(z,word_list_z,num_clusters = 10):
    """ Fit kmeans on the embedding representations and return a mapping of word to cluster indices. Please use the default values for
        the rest of the kmeans parameters.

        Arg: z: word embeddings (ndarray)
              word_list_z: List of tokens in the model (List) 
              num_clusters: Number of clusters (int)
        Returns:
            pre_trained_word2vec_model: word2vec model trained on the tokenized sentences.
            z2: word embeddings (ndarray)
            word_centroid_map_z: A mapping of word to cluster index it belongs to. (Dict)      

    """
    ### Add your code here.
    start = time.time() # Start time

    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit_predict(z)
    #print(kmeans.labels_.shape)
    word_centroid_map_z = dict(zip( word_list_z, kmeans ))
    ######################
    end = time.time()
    elapsed = end - start
    print ("Time taken for K Means clustering: ", elapsed, "seconds.")
    return word_centroid_map_z

#fit_kmeans(z1, word_list_z1, 10)

word_centroid_map_z1 = fit_kmeans(z1, word_list_z1, 10)

Time taken for K Means clustering:  0.7413699626922607 seconds.


In [0]:
def print_clusters(word_centroid_map_z, model_name):
    """ Print max(20, cluster_size) words for each of the clusters.

        Args: word_centroid_map_z: A mapping of word to cluster index it belongs to. (Dict)  
              model_name: Model Name (str)


    """
    print("The clusters for {0} are....".format(model_name))
    ### Add your code here.
    n=20
    num_clusters = max( word_centroid_map_z.values() ) + 1
    for cluster in range(num_clusters):
      words = []
      for i in range(len(word_centroid_map_z.values())):
        if (len(words)<=n):
          if( list(word_centroid_map_z.values())[i] == cluster ):
              words.append(list(word_centroid_map_z.keys())[i])
        else:
          break
      print (cluster," : ",words)

    ######################
print_clusters(word_centroid_map_z1, "model1")

The clusters for model1 are....
0  :  ['place', 'best', 'mexican', 'ever', 'new', 'favorite', 'area', 'many', 'find', 'places', 'thai', 'found', 'looking', 'valley', 'west', 'authentic', 'far', 'spot', 'restaurants', 'avondale', 'town']
1  :  ['like', 'pizza', 'little', 'menu', 'much', 'made', 'meal', 'small', 'bit', 'thing', 'ok', 'big', 'different', 'nothing', 'enough', 'large', 'full', 'used', 'usually', 'inside', 'thought']
2  :  ['chicken', 'ordered', 'sauce', 'fries', 'rice', 'salad', 'cheese', 'fried', 'meat', 'flavor', 'shrimp', 'bread', 'soup', 'tasted', 'tacos', 'sandwich', 'beans', 'beef', 'spicy', 'cooked', 'red']
3  :  ['get', 'one', 'would', 'even', 'never', 'could', 'make', 'know', 'give', 'sure', 'want', 'right', 'two', 'still', 'home', 'something', 'see', 'though', 'wanted', 'free', 'eating']
4  :  ['got', 'also', 'burger', 'hot', 'tried', 'taste', 'side', 'husband', 'salsa', 'chips', 'sweet', 'tea', 'wife', 'huge', 'served', 'perfect', 'cold', 'ice', 'wings', 'top', '

In [0]:
print(word_list_z1)
print(word_centroid_map_z1['good'])
print(model1["thai"])

['food', 'good', 'place', 'great', 'service', 'like', 'time', 'back', 'get', 'one', 'go', 'order', 'chicken', 'would', 'really', 'ordered', 'got', 'always', 'love', 'restaurant', 'delicious', 'us', 'try', 'friendly', 'also', 'nice', 'best', 'even', 'first', 'came', 'staff', 'pizza', 'never', 'well', 'little', 'fresh', 'sauce', 'come', 'went', 'definitely', 'fries', 'eat', 'could', 'rice', 'amazing', 'menu', 'better', 'salad', 'much', 'location', 'going', 'made', 'burger', 'said', 'lunch', 'wait', 'clean', 'minutes', 'customer', 'people', 'everything', 'make', 'know', 'meal', 'cheese', 'pretty', 'give', 'asked', 'times', 'mexican', 'ever', 'sure', 'way', 'bad', 'say', 'want', 'hot', 'new', 'family', 'tried', 'right', 'experience', 'two', 'fried', 'took', 'still', 'take', 'meat', 'taste', 'flavor', 'recommend', 'side', 'last', 'table', 'since', 'shrimp', 'favorite', 'manager', 'home', 'excellent', 'fast', 'every', 'drinks', 'awesome', 'next', 'think', 'told', 'server', 'husband', 'small'

In [0]:
clean_train_reviews = []
clean_test_reviews = []
### Add your code here.
def preprocess_review(review):
    """Helper function to clean the reviews.

     Arg: review: review text.
     Returns: clean_review : Cleaned reviews

     You should carry out the following steps.
     1. Remove HTML Tags.
     2. Remove non-letter characters.
     3. Convert to lower case.
     4. Remove stopwords.
    """

    #Write your code below.
    review_text = BeautifulSoup(review).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()
    stops = set(stopwords.words("english")) 
    meaningful_words = [w for w in words if not w in stops]
    clean_review= " ".join( meaningful_words )
    
    return clean_review

num_reviews = train["review"].size
for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( preprocess_review( train["review"][i] ) )

num_rev = len(test["review"])

# Append the reviews one by one after preprocessing
for i in range(0,num_rev):
    clean_review = preprocess_review( test["review"][i] )
    clean_test_reviews.append( clean_review )

#######################

In [0]:
def create_bag_of_centroids(tokenized_review, word_centroid_map, num_clusters = 10):
    """ Create a bag of kmeans centroids for each review i.e. for a review we return an array of length num_clusters with each 
        element of the array indicating how many words(tokens) of the review belong to that cluster.

        Args: tokenized_review: list of tokens corresponding to a review (List)
              word_centroid_map: word to cluster_index map for the model (Dict)
              num_clusters: Number of clusters (int)
        
        Returns: bag_of_centroids: An array containing the count of tokens in each cluster (ndarray)
    """
  
    ### Add your code here.
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    # num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_clusters, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in tokenized_review:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1    
    #######################
    return bag_of_centroids

In [0]:
def create_design_matrices(data, cleaned_reviews, word_centroid_map_z1, num_clusters = 10):
    """ Creates the design matrices X1(for trained model) and X2(for pretrained google word2vec model) for the given data

        Args: data: Train/Test data (pandas.core.frame.DataFrame)
              cleaned_reviews: List of tokenized reviews(sentences are not split and stopwords removed) (List)
              word_centroid_map_z1: word to cluster map for model1 (Dict)
              word_centroid_map_z2: word to cluster map for model2 (Dict)
              num_clusters: Number of KMeans clusters (int)

        Returns:
              x1_data: Design matrices X1-- Shape should be num_reviews*num_clusters (np.ndarray) 
              x2_data: Design matrices X2-- Shape should be num_reviews*num_clusters (np.ndarray)


    """
    ### Add your code here.
    x1_data = np.zeros( (data["text"].size, num_clusters),dtype="float32" )
    counter = 0
    for review in cleaned_reviews:
      x1_data[counter] = create_bag_of_centroids( review, word_centroid_map_z1 )
      counter += 1

    ######################
    return x1_data
x1_train = create_design_matrices(train,reviews_train_clean,word_centroid_map_z1,10)
x1_test = create_design_matrices(test,reviews_test_clean,word_centroid_map_z1,10)


In [0]:
x1_train.shape


(10161, 10)